## Import module

In [1]:
import math

importing Jupyter notebook from utils.ipynb


## Example of the network

In [2]:
"""
VGG13
VGG_image = [224, 224, 112, 112, 56, 56, 28, 28]
VGG_kernel = [3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
VGG_channel = [3, 64, 64, 128, 128, 256, 256, 512, 512]
n_layers = [1, 1, 1, 1, 1, 1, 1, 1]

image = VGG_image
kernel = VGG_kernel
channel = VGG_channel
"""

'\nVGG13\nVGG_image = [224, 224, 112, 112, 56, 56, 28, 28]\nVGG_kernel = [3, 3, 3, 3, 3, 3, 3, 3, 3, 3]\nVGG_channel = [3, 64, 64, 128, 128, 256, 256, 512, 512]\nn_layers = [1, 1, 1, 1, 1, 1, 1, 1]\n\nimage = VGG_image\nkernel = VGG_kernel\nchannel = VGG_channel\n'

In [3]:

"""
Resnet18
"""
Res_image = [112, 56, 28, 28, 14, 14, 7, 7]
Res_kernel = [7, 3, 3, 3, 3, 3, 3, 3]
Res_channel = [3, 64, 64, 128, 128, 256, 256, 512, 512]
n_layers = [1, 4, 2, 2, 2, 2, 2, 2]

image = Res_image
kernel = Res_kernel
channel = Res_channel

## Input the image size

In [1]:
image = list(map(int, input().split()))

 112 56 28 28 14 14 7 7


## Input the kernel size

In [2]:
kernel = list(map(int, input().split()))

 7 3 3 3 3 3 3 3


## Input the channel ( have to include the channel of first layer 3 )

In [3]:
channel = list(map(int, input().split()))

 3 64 64 128 128 256 256 512 512


## The number of convolutional layers

In [7]:
# n_layers = list(map(int, input().split()))

## Input the array size

In [8]:
array = list(map(int, input().split()))
# row ,col

 512 512


## Input the bit precision ( determines the number of utilized columns )

In [9]:
bit_precision = int(input())

 1


## Functions

In [10]:
def im2col (image_col, image_row, filter_col, filter_row, in_channel, out_channel, array_row, array_col, bit_precision) :

    real_used_col = math.floor(array_col / bit_precision)
    col_slide = image_col - filter_col + 1
    row_slide = image_row - filter_row + 1
    
    col_cycle = math.ceil(out_channel/real_used_col)
    row_cycle = math.ceil(filter_row*filter_col*in_channel/array_row)

    total_cycle = col_slide * row_slide * row_cycle * col_cycle
    
    return total_cycle

def SDK (image_col, image_row, filter_col, filter_row, in_channel, out_channel, \
                    array_row, array_col, bit_precision) :
    
    row_vector = filter_row * filter_col * in_channel
    col_vector = out_channel
    
    used_row = math.ceil(row_vector/array_row)
    used_col = math.ceil(col_vector/array_col)
    
    new_array_row = array_row * used_row
    new_array_col = array_col * used_col

    # initialize
    cycle = []
    w = []
    w.append(filter_row*filter_col)
    cycle.append(used_row*used_col*(image_row-filter_row+1)*(image_col-filter_col+1))
    
    i=0
    while True :
        i += 1
        pw_row = filter_row + i - 1 
        pw_col = filter_col + i - 1
        pw = pw_row * pw_col
        if pw*in_channel <= new_array_row and i * i * out_channel <= new_array_col :
            parallel_window_row = math.floor((image_row - (filter_row + i) + 1)/i) + 1
            parallel_window_col = math.floor((image_col - (filter_col + i) + 1)/i) + 1
            
            if parallel_window_row * parallel_window_row * used_row * used_col <= cycle[0] :
                del cycle[0]
                del w[0]
                cycle.append(parallel_window_row * parallel_window_col * used_row * used_col)
                w.append(pw)
            
        else :
            break
        
    
    return cycle, w

    
# ceil : up, floor : down
def vw_sdk (image_col, image_row, filter_col, filter_row, in_channel, out_channel, \
                    array_row, array_col, bit_precision) :
    real_used_col = math.floor(array_col / bit_precision)

    i = 0 # initialize # overlap col
    j = 1 # overlap row

    reg_total_cycle = [] # initialize
    reg_overlap_row = []
    reg_overlap_col = []
    reg_row_cycle = []
    reg_col_cycle = []
    reg_ICt = []
    reg_OCt = []
    
    while True :
        try :
            i += 1
            if (i + filter_col) > image_col : 
                i = 1
                j += 1
                if j + filter_row > image_row : 
                    break

            # for parallel_window computing
            reg_N_parallel_window_row = math.floor((image_row - (filter_row + i) + 1)/i) + 1
            reg_N_parallel_window_col = math.floor((image_col - (filter_col + j) + 1)/j) + 1
            
            # for cycle computing
            # Tiled IC
            if in_channel == 3 :
                ICt = math.floor(array_row /((filter_row + i - 1)*(filter_col + j - 1)))
                if ICt > in_channel :
                    ICt = 3
                row_cycle = math.ceil(in_channel / ICt)
            else :
                ICt = math.floor(array_row /((filter_row + i - 1)*(filter_col + j - 1)))
                row_cycle = math.ceil(in_channel / ICt)
            
            # Tiled OC
            OCt =  math.floor(real_used_col / (i * j))
            col_cycle = math.ceil(out_channel / OCt)
    
            reg_N_of_computing_cycle = reg_N_parallel_window_row * reg_N_parallel_window_col \
                                    * row_cycle * col_cycle
            
            if i == 1 : # initialize
                reg_total_cycle.append(reg_N_of_computing_cycle)
                reg_overlap_row.append(i)
                reg_overlap_col.append(j)
                reg_row_cycle.append(row_cycle)
                reg_col_cycle.append(col_cycle)
                reg_ICt.append(ICt)
                reg_OCt.append(OCt)

            if reg_total_cycle[0] > reg_N_of_computing_cycle :
                del reg_total_cycle[0]
                del reg_overlap_row[0]
                del reg_overlap_col[0]
                del reg_row_cycle[0]
                del reg_col_cycle[0]
                del reg_ICt[0]
                del reg_OCt[0]

                reg_total_cycle.append(reg_N_of_computing_cycle)
                reg_overlap_row.append(i)
                reg_overlap_col.append(j)
                reg_row_cycle.append(row_cycle)
                reg_col_cycle.append(col_cycle)
                reg_ICt.append(ICt)
                reg_OCt.append(OCt)

    
        except ZeroDivisionError :
            continue

    return reg_total_cycle[0], reg_overlap_col[0], reg_overlap_row[0], reg_row_cycle[0], reg_col_cycle[0], reg_ICt[0], reg_OCt[0] 
def network_information() :
    print("="*50)
    print( " Array Size = {} x {}".format(array[0], array[1]))
    print( " Bit precision = {}".format(bit_precision))
    print( " Actual Array Size = {} x {}".format(array[0], int(array[1]/bit_precision)))
    print("-"*30)
    
    print(" NETWORK INFORMATION ")
    print("-"*30)
    for i in range(len(image)) :
        print(" CONV LAYER "+ str(i+1))
        print("    Image   Size = {} x {}".format(image[i], image[i]))
        print("    Kernel  Size = {} x {}".format(kernel[i], kernel[i]))
        print("    Channel Size = {} x {}".format(channel[i], channel[i+1]))
    print("="*50)
    
def result () :
    CC_im2col = []
    
    CC = []
    SDK_height = []
    SDK_width = []
    AR_cycle = []
    AC_cycle = []
    IC_tiled = []
    OC_tiled = []
    
    CC_SDK = []
    PW_SDK = []
    
    for i in range(len(image)) :
        T_im2col = im2col(image[i], image[i], kernel[i], kernel[i], channel[i], channel[i+1], array[0], array[1], bit_precision)
        CC_im2col.append(T_im2col)

        T_SDK, w = SDK(image[i], image[i], kernel[i], kernel[i], channel[i], channel[i+1], array[0], array[1], bit_precision)
        CC_SDK.append(T_SDK[0])
        PW_SDK.append(w[0])

        T_cycle, SDK_h, SDK_w, ARC, ACC, tiled_IC, tiled_OC = vw_sdk(image[i], image[i], kernel[i], kernel[i], channel[i], channel[i+1], array[0], array[1], bit_precision)
        CC.append(T_cycle)
        SDK_height.append(SDK_h)
        SDK_width.append(SDK_w)
        AR_cycle.append(ARC)
        AC_cycle.append(ACC)
        IC_tiled.append(tiled_IC)
        OC_tiled.append(tiled_OC)
    
#     total_im2col = 0
#     total_SDK = 0
#     total_VWSDK = 0
    
    print("="*50)
    print(" RESULTS of COMPUTING CYCLES")
    print("-"*30)
    for i in range(len(image)) :
        print(" CONV LAYER "+ str(i+1))
        print("    Im2col = {}".format(CC_im2col[i]))
        print("    SDK    = {}".format(CC_SDK[i]))
        print("      - shape of PW = {} x {} x {} x {}".format(int(math.sqrt(PW_SDK[i])), int(math.sqrt(PW_SDK[i])), channel[i], channel[i+1]))
        print("    VW-SDK = {}".format(CC[i]))
        print("      - Optimal shape of PW = {} x {} x {} x {}".format(kernel[i] + SDK_width[i]-1, kernel[i] + SDK_height[i]-1, IC_tiled[i], OC_tiled[i]))
        print("      - Reduction Compared to Im2col = {:.2f} %".format((CC_im2col[i]-CC[i])/CC_im2col[i]*100))
        print("      - Reduction Compared to SDK    = {:.2f} %".format((CC_SDK[i]-CC[i])/CC_SDK[i]*100))    
    print("="*50)

## Results

In [11]:
network_information()
result()

 Array Size = 512 x 512
 Bit precision = 1
 Actual Array Size = 512 x 512
------------------------------
 NETWORK INFORMATION 
------------------------------
 CONV LAYER 1
    Image   Size = 112 x 112
    Kernel  Size = 7 x 7
    Channel Size = 3 x 64
 CONV LAYER 2
    Image   Size = 56 x 56
    Kernel  Size = 3 x 3
    Channel Size = 64 x 64
 CONV LAYER 3
    Image   Size = 28 x 28
    Kernel  Size = 3 x 3
    Channel Size = 64 x 128
 CONV LAYER 4
    Image   Size = 28 x 28
    Kernel  Size = 3 x 3
    Channel Size = 128 x 128
 CONV LAYER 5
    Image   Size = 14 x 14
    Kernel  Size = 3 x 3
    Channel Size = 128 x 256
 CONV LAYER 6
    Image   Size = 14 x 14
    Kernel  Size = 3 x 3
    Channel Size = 256 x 256
 CONV LAYER 7
    Image   Size = 7 x 7
    Kernel  Size = 3 x 3
    Channel Size = 256 x 512
 CONV LAYER 8
    Image   Size = 7 x 7
    Kernel  Size = 3 x 3
    Channel Size = 512 x 512
 RESULTS of COMPUTING CYCLES
------------------------------
 CONV LAYER 1
    Im2col = 112